In [6]:
import pandas as pd
import numpy as np
import json
import requests

In [28]:
!pip install plotly

  Using cached plotly-5.11.0-py2.py3-none-any.whl (15.3 MB)
  Using cached tenacity-8.1.0-py3-none-any.whl (23 kB)


In [52]:
import requests

def bitqueryAPICall(query: str):  
    headers = {'X-API-KEY': 'API-KEY'}
    request = requests.post('https://graphql.bitquery.io/',
                            json={'query': query, 'variables': variables}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed and return code is {}.      {}'.format(request.status_code,query))


# The GraphQL query

query = """
{
  ethereum(network: ethereum) {
    dexTrades(options: {limit: 10000, asc: "timeInterval.minute"}, 
      protocol: {is: "Uniswap v2"}, 
      date: {after: "2020-10-10"}
      buyCurrency: {is: "0xdac17f958d2ee523a2206206994597c13d831ec7"}, 
      sellCurrency: {is: "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"}) {
      timeInterval {
        minute(count: 30)
      }
      buyCurrency {
        symbol
        address
      }
      buyAmount
      sellCurrency {
        symbol
        address
      }
      sellAmount
      trades: count
      maximum_price: price(calculate: maximum)
      minimum_price: price(calculate: minimum)
      open_price: minimum(of: block, get: price)
      close_price: maximum(of: block, get: price)
    }
  }
}
"""

variables = {
  "network": "bitcoin",
  "token": "18cBEMRxXHqzWWCxZNtU91F5sbUNKhL5PX"
}

result = bitqueryAPICall(query) 





In [ ]:
##result['data']['ethereum']['dexTrades'][2]['timeInterval']['minute']
##result['data']['ethereum']['dexTrades'][0]

In [53]:
df = pd.DataFrame(data=result['data']['ethereum']['dexTrades'])

In [54]:
for i in range(len(df)):
    df['timeInterval'][i] = result['data']['ethereum']['dexTrades'][i]['timeInterval']['minute']

/var/folders/zs/v8gw8g596vjbsqn51ryrf4b00000gn/T/ipykernel_8459/1327823660.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [55]:
len(df)

10000

In [56]:
import plotly.graph_objects as go

import pandas as pd
from datetime import datetime


fig = go.Figure(data=[go.Candlestick(x=df['timeInterval'],
                open=df['open_price'],
                high=df['maximum_price'],
                low=df['minimum_price'],
                close=df['close_price'])])

fig.update_layout(
    colorway=["#000000", "#000000"],
    template="plotly_dark",
)
